### Let's scrape fake news with corresponding labels and create a  dataset. 

In [1]:
#Imports

import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from newspaper import Article
import newspaper
pd.set_option('max_colwidth', 100)

In [2]:
#Let's scrape the list of narratives and corresponding links with the Beautiful Soup library.

URL = 'https://rusdisinfo.voxukraine.org'
content = requests.get(URL).text
soup = BeautifulSoup(content, "html.parser")
publications = soup.find(class_='Home_container__bCOhY')
keys = publications.find_all('h2', class_ = "FakeItem_fakeHeading__Uc0QN")
values = publications.find_all('a')
valid_links = []
for v in values:
    link = URL + v.get('href')
    r = requests.get(link)
    if r.status_code != 404:
        valid_links.append(link)

labeled_dict = {}

for index, k in enumerate(keys):
    k = keys[index].text[2:]
    v = valid_links[index]      
    labeled_dict.update({k:v})

for k,v in labeled_dict.items():
    print(k,v)

Maidan in 2014 was a coup d'état https://rusdisinfo.voxukraine.org/narratives/Maidan%20in%202014%20was%20a%20coup%20d'%C3%A9tat
Nazis and ultra-nationalists in Ukraine https://rusdisinfo.voxukraine.org/narratives/Nazis%20and%20ultra-nationalists%20in%20Ukraine
Nord Stream-2 https://rusdisinfo.voxukraine.org/narratives/Nord%20Stream-2
Russia needs to protect itself https://rusdisinfo.voxukraine.org/narratives/Russia%20needs%20to%20protect%20itself
Ukraine is an illiberal state https://rusdisinfo.voxukraine.org/narratives/Ukraine%20is%20an%20illiberal%20state
Ukraine is a failed state https://rusdisinfo.voxukraine.org/narratives/Ukraine%20is%20a%20failed%20state
The West controls Ukraine and uses it to its advantage https://rusdisinfo.voxukraine.org/narratives/The%20West%20controls%20Ukraine%20and%20uses%20it%20to%20its%20advantage
The West is not interested in dealing with Ukraine, moreover, solving its problems. https://rusdisinfo.voxukraine.org/narratives/The%20West%20is%20not%20inter

In [3]:
#Now let's click on each link with the help of Selenium and scrape the news inside of it.

DRIVER_PATH = '/path/to/chromedriver'
driver = webdriver.Chrome(executable_path=DRIVER_PATH)
driver.get('https://google.com')

dictionary_news = {}

for k,v in labeled_dict.items():
    driver.get(v)
    all_links = driver.find_elements(By.CLASS_NAME, 'Narrative_fakeLink___YbTe')
    for i in all_links:
        i.click()
        
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    list_dirty = soup.find_all(class_ = 'Narrative_media__B2gNZ')
    final_list = []
    for u in list_dirty:
        link_news = u.find('a').get('href')
        final_list.append(link_news)

    dictionary_news.update({k:final_list})

/var/folders/4b/j_t85q4j2klgsp_lqvw39xk40000gn/T/ipykernel_3948/3127159200.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=DRIVER_PATH)


In [4]:
#Newspaper3k is a Python library for extracting & curating articles.
#Let's use it in order to create a labeled dataset of the news. 

scraped_news = []

for label, links in dictionary_news.items():
    
    # Current limit to 5 news from each category
    for url in links[:5]:
        article = newspaper.Article(url=url)
        article.download()

        try:
            article.parse()
            article ={"title": str(article.title), "text": str(article.text)}
            article.update({"link":url})
            article.update({"label": label})
            scraped_news.append(article)

        except:
            continue

In [6]:
df_news = pd.DataFrame(scraped_news)
df_news

,title,text,link,label
0,Krieg im Donbass: Ukraine verkündet das Ende des Abkommens von Minsk – Anti-Spiegel,"Präsident Selensky hat de facto das Ende des Abkommens von Minsk verkündet mitteilen lassen, sei...",https://www.anti-spiegel.ru/2021/krieg-im-donbass-ukraine-verkuendet-das-ende-des-abkommens-von-...,Maidan in 2014 was a coup d'état
1,Die Lebkuchenhexe aus Washington – Jetzt intrigiert sie gegen Nord Stream 2,Die Lebkuchenhexe aus Washington – Jetzt intrigiert sie gegen Nord Stream 2\n\nDa ist sie wieder...,https://de.rt.com/meinung/129862-lebkuchenhexe-im-weissen-haus/,Maidan in 2014 was a coup d'état
2,Spannungen verschärfen sich: Ukraine laut Selenskyj „bereit“ für Krieg mit Russland,"Während sich die Spannungen in der Schwarzmeerregion weiter verschärfen, erklärte der ukrainisch...",https://www.wsws.org/de/articles/2021/04/23/ukra-a23.html,Maidan in 2014 was a coup d'état
3,Russland und China versus USA mit NATO-Verbündeten: Vorbereitungen auf den Ernstfall laufen,Russland und China versus USA mit NATO-Verbündeten: Vorbereitungen auf den Ernstfall laufen\n\nI...,https://de.rt.com/meinung/128478-russland-und-china-versus-usa/,Maidan in 2014 was a coup d'état
4,Die NATO rückt in der Ukraine immer näher an einen Krieg gegen Russland,"Was die Ukraine anbelangt, so scheint Ankara das Tempo für die verstärkte Einmischung der NATO i...",https://uncutnews.ch/die-nato-rueckt-in-der-ukraine-immer-naeher-an-einen-krieg-gegen-russland/,Nazis and ultra-nationalists in Ukraine
5,Bielorussia nel mirino dei barbari di Washington: cosa succede?,Ciò che sta accadendo in questi giorni in Bielorussia è la fotocopia esatta di quanto è già acca...,https://www.radioradio.it/2020/08/bieloRussia-nel-mirino-dei-barbari-di-washington-cosa-succede/,Nazis and ultra-nationalists in Ukraine
6,Die US-Kriegspläne für die Ukraine,"Offen gesagt, Bidens Ansprache an die US-Nation (die erste in seinem Leben) war wahrscheinlich d...",https://uncutnews.ch/die-us-kriegsplaene-fuer-die-ukraine/,Nazis and ultra-nationalists in Ukraine
7,Bloomberg – La Germania non ha promesso agli Stati Uniti di sospendere Nord Stream 2 a causa del...,La Germania ha negato di aver promesso agli Stati Uniti di bloccare il gasdotto ‘Nord Stream 2’ ...,https://resetitaliablog.altervista.org/bloomberg-la-germania-non-ha-promesso-agli-stati-uniti-di...,Nord Stream-2
8,Il prezzo del gas resta a livelli record,"Il prezzo del gas resta a livelli record\n\nTra le ragioni dell’impennata: il freddo, la crisi u...",https://www.corrierenazionale.net/il-prezzo-del-gas-resta-a-livelli-record/,Nord Stream-2
9,Il prezzo del gas resta a livelli record,"Il prezzo del gas resta a livelli record\n\nTra le ragioni dell’impennata: il freddo, la crisi u...",https://www.corrierenazionale.net/il-prezzo-del-gas-resta-a-livelli-record/,Nord Stream-2


In [11]:
df_news.to_csv('Scraped_news.csv')